In [33]:
import os
import ifcopenshell
import pandas as pd

# Directory where the IFC files are located
directory_path = os.getcwd()
print(f"Directory: {directory_path}")

# Initialize lists to store the data
data = []

# Loop through all subfolders of the directory
for root, dirs, files in os.walk(directory_path):
    # Loop through all files in the current subfolder
    for file in files:
        # Check if the file has a .ifc extension
        if file.endswith('.ifc'):
            file_path = os.path.join(root, file)
            print(f"Processing {file_path}")
            
            ifc_file = ifcopenshell.open(file_path)
            entities = ifc_file.by_type("IfcProduct")
            print(f"Found {len(entities)} entities in {file}")

            for entity in entities:
                if hasattr(entity, 'IsDefinedBy'):
                    for definition in entity.IsDefinedBy:
                        if definition.is_a('IfcRelDefinesByProperties'):
                            property_set = definition.RelatingPropertyDefinition
                            
                            if hasattr(property_set, 'HasProperties'):
                                for property in property_set.HasProperties:
                                    attribute_value = property.NominalValue.wrappedValue if hasattr(property, 'NominalValue') and property.NominalValue is not None else "N/A"
                                    entity_data = {
                                        'File': file,
                                        'Entity': entity.is_a(),
                                        'Attribute': property.Name,
                                        'Value': attribute_value,
                                        'GUID': entity.GlobalId
                                    }
                                    data.append(entity_data)
                                    #print(f"Data added for entity {entity.GlobalId}")


# Convert the list into a pandas dataframe
df = pd.DataFrame(data)

# Group the dataframe by GUID and Attribute
grouped = df.groupby(['GUID', 'Attribute']).agg({'Value': 'first'})
grouped = grouped.unstack()
grouped.columns = grouped.columns.droplevel()


# Save the updated dataframe to an Excel file
excel_file_path = os.path.join(directory_path, "datadrop.xlsx")
grouped.to_excel(excel_file_path, index=False)

print("DATADROP - SUCCESS!")


Directory: C:\WinPython\notebooks\RB_Jupiter for BIM
Processing C:\WinPython\notebooks\RB_Jupiter for BIM\VK_IKT29_ET_Revit.ifc
Found 1814 entities in VK_IKT29_ET_Revit.ifc
DATADROP - SUCCESS!
